#TS-CausalNN applied to Synthetic Dataset-3


This notebook contains the proposed model. Here we have developed the proposed Causal Conv2D layer and the optimization function.

The functions to visualize the predicted causal graph are available after the model training codes. The predicted graph is compared with ground truth using an adjacency matrix (array).   

In this notebook, we applied the proposed model to the synthetic dataset-3 to generate a full causal graph and summary graph.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
np.random.seed(101)
T = 5000
int_T = np.expand_dims(np.arange(T), axis=1)
x1 = np.random.randn(T,1)
noise1 = 0.5*np.random.randn(T,1)
#source1[1] = noise[1] + 10
#source1[2] = noise[2] + 10
#for x in range(3,t):
#  source1[x] = 0.125*math.sqrt(2)*math.exp(-source1[x-1]*source1[x-1]/2) + noise[x]

for i in range(0,T):
  if(i<5):
    x1[i] = x1[i] #*0.1 #+ 1.5*np.sin(i/50)
  else:
    x1[i] = 0.5*x1[i-5] + 0.5*x1[i-2] + noise1[i]
x2 = 0.8*x1 + 1.5*np.sin(int_T/50) + 0.5*np.random.randn(T,1)
noise2 = 0.5*np.random.randn(T,1)
for i in range(1,T):
  x2[i] = 0.1*x1[i] + 0.7*x1[i-1] + 1.5*np.sin(int_T[i]/50)+noise2[i]

x3 = 0.8*x1 + 0.5*np.random.randn(T,1)
noise3 = 0.5*np.random.randn(T,1)
for i in range(1,T):
  x3[i] = 0.8*x1[i-1] +noise3[i]


x4 = 0.8*x3 + (np.sin(int_T/50)+np.sin(int_T/20)) + 0.5*np.random.randn(T,1)
noise4 = 0.5*np.random.randn(T,1)
for i in range(1,T):
  x4[i] = 0.2*x4[i-1] + 0.4*x3[i] + 0.4*x3[i-1] + 0.4*x1[i-1] + (np.sin(int_T[i]/50)+np.sin(int_T[i]/20)) +noise4[i]

#Data = [x1,x2,x3,x4];
dict={'S1':x1.reshape(-1),'S2':x2.reshape(-1),'S3':x3.reshape(-1),'S4':x4.reshape(-1)}
data=pd.DataFrame(dict)

In [ ]:
data.to_csv('synthetic_3rd_dataset.csv',header=True,index=False)
from google.colab import files
files.download("synthetic_3rd_dataset.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
data.iloc[-100:,:]

,S1,S2,S3,S4
4900,0.899602,1.506833,2.246201,3.145405
4901,2.831144,0.541263,1.301042,1.972708
4902,1.308382,1.368133,2.411568,2.099916
4903,2.459179,0.281900,0.661700,0.760705
4904,1.776536,0.174835,1.988398,0.932680
...,...,...,...,...
4995,1.956832,0.474266,0.970847,-0.537619
4996,1.739651,0.932630,2.419024,-0.206812
4997,2.233976,0.248079,0.180797,0.486272
4998,2.546138,2.043770,2.058555,0.281335


In [ ]:
import scipy.io

def signaltonoise(a, axis=0, ddof=0):
    a = np.asanyarray(a)
    m = a.mean(axis)
    sd = a.std(axis=axis, ddof=ddof)
    return np.where(sd == 0, 0, (m*m)/(sd*sd))



In [ ]:
data_np = data.to_numpy()

In [ ]:
snr1 = signaltonoise(data_np[:,0])
snr2 = signaltonoise(data_np[:,1])
snr3 = signaltonoise(data_np[:,2])
snr4 = signaltonoise(data_np[:,3])

In [ ]:
snr1, snr2, snr3, snr4

(array(0.38524771), array(0.27584791), array(0.36343834), array(0.32932265))

In [ ]:
true_graph = np.zeros((4,4))
true_graph[0,0]=1
true_graph[0,1]=1
true_graph[0,2]=1
true_graph[0,3]=1
true_graph[2,3]=1
true_graph[3,3]=1
true_graph

array([[1., 1., 1., 1.],
       [0., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.]])

In [ ]:
true_full_graph = np.zeros((4,24))
true_full_graph[0,0]=1
true_full_graph[0,12]=1

true_full_graph[1,16]=1
true_full_graph[1,20]=1

true_full_graph[2,16]=1

true_full_graph[3,16]=1
true_full_graph[3,18]=1
true_full_graph[3,19]=1
true_full_graph[3,22]=1
true_full_graph


array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 1., 1., 0., 0., 1., 0.]])

In [ ]:
!pip install cdt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 921.1/921.1 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7394 sha256=33664621d478b15fb2aa106b86b66743bf8d93f96d38cdf9da334c2e4861a9e0
  Stored in directory: /root/.cache/pip/wheels/a9/8a/bd/81082387151853ab8b6b3ef33426e98f5cbfebc3c397a9d4d0
  Created wheel for skrebate: filename=skrebate-0.62-py3-none-any.whl size=29255 sha256=6c09944c9c9e36a6abcd4877474310588c25f6a4d09c7d2877c4032527665416
  Stored in directory: /root/.cache/pip/wheels/dd/67/40/683074a684607162bd0e34dcf7ccdfcab5861c3b2a83286f3a
Successfully built GPUtil skrebate


In [ ]:
import cdt

No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.


#Data Pre-processing

In [ ]:
import numpy as np
import pandas as pd

def data_preprocessing(data, max_lag=5):
  from sklearn.preprocessing import MinMaxScaler

  #convert to numpy array
  syn_data_np = data.to_numpy()

  #normalize the dataset
  scaler_X = MinMaxScaler(feature_range=(0, 1))
  syn_data_np_nor = scaler_X.fit_transform(syn_data_np)
  syn_data_np = syn_data_np_nor

  #transform into 2D data
  syn_data_np_T= syn_data_np.T
  syn_data_pro = np.zeros((syn_data_np.shape[0]-max_lag,syn_data_np.shape[1],(max_lag+1)))
  for i in range(0, (syn_data_np.shape[0]-max_lag)):
    syn_data_pro[i,:,:]= syn_data_np_T[:, i:i+(max_lag+1)]
  syn_data_2d = np.expand_dims(syn_data_pro, axis =-1)

  #make datafrom with normalized data
  syn_data_nor_df =pd.DataFrame(data = syn_data_np,
                  columns = data.columns)

  #transform normalized data into 1D shape with lagged and current time values
  size_1d = syn_data_np.shape[1]*(max_lag+1)
  print(size_1d)
  syn_data_1d = np.zeros((syn_data_np.shape[0]-max_lag,size_1d))
  for i in range(0, (syn_data_np.shape[0]-max_lag)):
    for j in range(0,(max_lag+1)):
      j_end = j * syn_data_np.shape[1]
      syn_data_1d[i,j_end:j_end+syn_data_np.shape[1]]= syn_data_np[i+j, :]

  #transform non-normalized data into 1D shape with lagged and current time values
  syn_data_np_2 = data.to_numpy()
  syn_data_1d_not_norm = np.zeros((syn_data_np_2.shape[0]-max_lag,size_1d))
  for i in range(0, (syn_data_np_2.shape[0]-max_lag)):
    for j in range(0,(max_lag+1)):
      j_end = j * syn_data_np_2.shape[1]
      syn_data_1d_not_norm[i,j_end:j_end+syn_data_np_2.shape[1]]= syn_data_np_2[i+j, :]


  return syn_data_np_nor, syn_data_2d, syn_data_nor_df, syn_data_1d,  syn_data_1d_not_norm

In [ ]:
syn_data_np, syn_data_2d, syn_data_nor_df, syn_data_1d, syn_data_1d_not_norm = data_preprocessing(data, max_lag=5)

In [ ]:
syn_data_2d.shape

(4995, 4, 6, 1)

In [ ]:
syn_data_np.shape

(5000, 4)

In [ ]:
syn_data_np.shape, syn_data_2d.shape

((5000, 4), (4995, 4, 6, 1))

In [ ]:
data_y_syn = syn_data_np[5:,0:4]
data_y_syn.shape

(4995, 4)

# Model

In [ ]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten, Dropout, AveragePooling2D, LSTM, Activation, ConvLSTM2D, TimeDistributed, Input, Reshape
from keras.layers import UpSampling1D, Conv2DTranspose, UpSampling2D, Conv1D, AveragePooling1D, LeakyReLU
from keras.models import Model
from keras.optimizers import SGD
from keras import callbacks
import keras.backend as K
import tensorflow as tf
from keras.layers import concatenate
from keras.regularizers import l1, l2
from time import time

keras.utils.set_random_seed(1001)

In [ ]:
class CausalConv2D(tf.keras.layers.Layer):
    def __init__(self, num_outputs, counter, *args, **kwargs):
        super(CausalConv2D, self).__init__()
        self.conv2d = tf.keras.layers.Conv2D(*args, **kwargs)
        self.num_outputs = num_outputs
        self.counter = counter

    def build(self, input_shape):
        self.W=self.add_weight(name='kernel',
                           shape=(input_shape[1], input_shape[2],input_shape[3], 1),
                           initializer = keras.initializers.RandomUniform(minval=0.05, maxval=0.5),
                           #initializer ='uniform',
                           #initializer = tf.keras.initializers.glorot_uniform(seed=8),
                           trainable=True)
        self.mask = np.ones(shape=self.W.shape)
        #print(self.W)
        self.mask[self.counter,(input_shape[2]-1),...] = 0.0

    #def masked_convolution_op(self, filters, kernel, mask):
    #    return self._convolution_op(filters, tf.math.multiply(kernel, tf.reshape(mask, mask.shape + [1,1] )))

    def get_weights(self):
        return super().get_weights()

    def call(self, inputs):
        self.W.assign(tf.math.multiply(self.W, self.mask))
        #self.conv2d._convolution_op = functools.partial(self.masked_convolution_op, mask=mask)
        #return self.conv2d.call(x)
        return self.conv2d.convolution_op(inputs, self.W)

In [ ]:
# 2D
def get_model_2d(input_dims):
    input_batch = Input(shape = input_dims)

    conv_model = Sequential()
    conv_model = Conv2D(filters=4, kernel_size=1, strides=(1,1), padding='valid', activation="linear", name='conv1')(input_batch)
    #conv_model = AveragePooling2D(pool_size=(1,1), strides=None, padding='valid', name='pool1')(conv_model) activation=LeakyReLU(0.05)
    conv_model = tf.math.reduce_mean(conv_model, axis=-1)
    conv_model = Reshape((4, 6, 1))(conv_model)
    #conv_model = Flatten()(conv_model )
    pooled_outputs = []
    for i in range(0, 4):
      #layer = CausalConv2D(num_outputs=1, counter=i, name="parr"+str(i))(conv_model) # , kernel_regularizer = l1(0.2)
      layer = CausalConv2D(filters=1, kernel_size=(4,6), num_outputs=1, counter=i, padding='valid', activation="tanh",)(conv_model)
      #den1 = layer(tf.ones([481, 30]))
      #conv = Conv2D(1, kernel_size=filter_sizes[i], padding='valid', activation='relu')(conv_model)
      pooled_outputs.append(layer)
    output = concatenate(pooled_outputs)
    output = Flatten()(output)

    model = Model(inputs=input_batch, outputs=output, name='cpred')

    return model

In [ ]:
# with gradient tape

class CausalNNModel(object):
    def __init__(self,
                 dims,
                 alpha=0.0,
                 rho = 1.0,
                 rho_max = 10e20,
                 h_tol = 1e-8,
                 init='glorot_uniform'):

        super(CausalNNModel, self).__init__()

        self.dims = dims
        self.n_stacks = len(self.dims) - 1
        self.alpha = alpha
        self.rho = rho
        self.h_p = np.Inf
        self.rho_max = rho_max
        self.h_tol = h_tol
        self.model_2d = get_model_2d(self.dims)
        print("====Model created=====")

        self.model = Model(inputs=self.model_2d.input, outputs=self.model_2d.output)


    def extract_features(self, x):
        return self.model_cnn.predict(x)

    def predict(self, x):  # predict cluster labels using the output of clustering layer
        q = self.model.predict(x, verbose=0)[1]
        return q.argmax(1)

    def custom_loss_function(self, y_true, y_pred):
      mse = keras.losses.mean_squared_error(y_true, y_pred)
      h_val = self.causal_loss_h()
      h_loss = 0.5 * self.rho * h_val * h_val + self.alpha * h_val
      lambda1 = 0.1
      adj_mat = self.get_mat()
      sparse_loss = lambda1 * tf.math.reduce_sum(tf.abs(adj_mat))
      #neg_weight = np.sum(adj_mat, where=adj_mat<0)
      #neg_loss = 0.5 * tf.abs(neg_weight)
      print('MSE Loss is: {}, h Loss is: {}, L1 loss: {}, Total Loss is: {}'.format(tf.reduce_mean(mse), h_loss, sparse_loss, tf.reduce_mean(mse)+h_loss))
      return mse + h_loss + sparse_loss #+ neg_loss

    def causal_loss_h(self):
      mat = self.get_mat()
      h_val = self.h_acy_1(mat[:, 20:])
      return h_val

    def get_mat(self):
      w1_2d_s = self.model.get_layer(index=-6).get_weights()
      w2_2d_s = self.model.get_layer(index=-5).get_weights()
      w3_2d_s = self.model.get_layer(index=-4).get_weights()
      w4_2d_s = self.model.get_layer(index=-3).get_weights()
      arr1_2d_s = np.expand_dims(np.squeeze(np.array(w1_2d_s), axis=(0,3,4)).flatten('F'), axis=0)
      arr2_2d_s = np.expand_dims(np.squeeze(np.array(w2_2d_s), axis=(0,3,4)).flatten('F'), axis=0)
      arr3_2d_s = np.expand_dims(np.squeeze(np.array(w3_2d_s), axis=(0,3,4)).flatten('F'), axis=0)
      arr4_2d_s = np.expand_dims(np.squeeze(np.array(w4_2d_s), axis=(0,3,4)).flatten('F'), axis=0)
      mat_2d_s = np.concatenate((arr1_2d_s, arr2_2d_s, arr3_2d_s, arr4_2d_s))
      #print(mat_2d_s)
      return mat_2d_s

    def h_acy_1(self, A):
      n_var = A.shape[0]
      h = tf.linalg.trace(tf.linalg.expm(A * A)) - n_var
      return h


    def h_acy(self, A):
      '''Calculate the constraint of A ensure that it's a DAG'''
      #(Yu et al. 2019 DAG-GNN)
      # h(w) = tr[(I + kA*A)^n_variables] - n_variables
      n_var = A.shape[0]
      M = tf.eye(n_var, num_columns = n_var) + A/n_var
      E = M
      for _ in range(n_var - 2):
        E = tf.linalg.matmul(E, M)
      h = tf.math.reduce_sum(tf.transpose(E) * M) - n_var
      return h

    def compile(self, optimizer='adam'):
        self.model.compile(optimizer=optimizer, loss= self.custom_loss_function) # ['mse', self.causal_loss()])

    def fit(self, x, y=None, maxiter=100, batch_size=512, save_dir='./results/temp'):
        t1 = time()


        # Step 2: deep clustering
        # logging file
        import csv
        logfile = open(save_dir + '/causalnn_log.csv', 'w')
        logwriter = csv.DictWriter(logfile, fieldnames=['iter','loss'])
        logwriter.writeheader()
        train_loader = tf.data.Dataset.from_tensor_slices((x, y)).batch(batch_size)
        optimizer = tf.keras.optimizers.Adam(1e-2)
        w1_2d_s = self.model.get_layer(index=-6).get_weights()
        arr1_2d_s = np.expand_dims(np.squeeze(np.array(w1_2d_s), axis=(0,3,4)).flatten('F'), axis=0)


        for epoch in range(int(maxiter)):
          print('Epoch: {}', epoch)
          h_n = None
          for (x, y) in train_loader:
            #eval loss and compute gradients
            with tf.GradientTape() as tape:
              tape.watch(self.model.trainable_variables)
              #passing through neural network
              output = self.model(x)
              #calculate loss
              loss = self.custom_loss_function(y, output)
              gradients = tape.gradient(loss, self.model.trainable_variables)
              optimizer.apply_gradients(zip(gradients, self.model.trainable_variables))
              h_n = self.causal_loss_h()
              #print('New h_val is :', h_n)
              #if h_n > 0.25 * self.h_p:
              #  self.rho = self.rho*10
              #else:
              #  break

          if h_n > 0.25 * self.h_p:
                self.rho = self.rho*1.1
          self.h_p = h_n
          print('New h_val is :', h_n)
          self.alpha += self.rho * self.h_p

          if self.h_p <= self.h_tol or self.rho >= self.rho_max:
            print('Before the loop end # h_val is: {}, rho is: {}'.format(self.h_p, self.rho))
            break

        #for ite in range(int(maxiter)):
        #  print('Epoch: {}', ite)
        #  self.model.fit(x, y, epochs=1, batch_size=batch_size, verbose=True)

        # save the trained model
        logfile.close()
        file_name  = "/CausalNN_model_final_" + str(round(time()))+ ".h5"
        print('saving model to:', save_dir + file_name)
        self.model.save_weights(save_dir + file_name)

        w1_2d_s_1 = self.model.get_layer(index=-6).get_weights()
        arr1_2d_s_1 = np.expand_dims(np.squeeze(np.array(w1_2d_s_1), axis=(0,3,4)).flatten('F'), axis=0)

        y_pred = self.model.predict(x)
        adj_mat = self.get_mat()

        print('The conv layer 1 weights before training :', arr1_2d_s)
        print('The conv layer 1 weights after training :', arr1_2d_s_1)

        return y_pred, adj_mat

#Model creation and training:

In [ ]:
keras.utils.set_random_seed(1001)

cnnmodel = CausalNNModel(dims=syn_data_2d.shape[1:])
cnnmodel.model.summary()
cnnmodel.compile()
y_pred, mat = cnnmodel.fit(x=syn_data_2d, y=data_y_syn, maxiter=20, batch_size=512, save_dir='/content/')

====Model created=====
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 4, 6, 1)]            0         []                            
                                                                                                  
 conv1 (Conv2D)              (None, 4, 6, 4)              8         ['input_1[0][0]']             
                                                                                                  
 tf.math.reduce_mean (TFOpL  (None, 4, 6)                 0         ['conv1[0][0]']               
 ambda)                                                                                           
                                                                                                  
 reshape (Reshape)           (None, 4, 6, 1)              0         ['t

In [ ]:
mat.shape

(4, 24)

In [ ]:
mat_df_2d_s = pd.DataFrame(mat)

In [ ]:
mat_f_df = pd.DataFrame(mat).T

In [ ]:
mat_df_2d_s.T

,0,1,2,3
0,0.060424,-0.086885,0.158718,-0.023992
1,-0.071343,0.219383,-0.102552,-0.049103
2,0.197360,0.230671,0.013068,-0.011465
3,-0.518587,0.195263,0.023268,-0.004621
4,0.416361,0.019698,0.006211,0.048413
5,-0.010432,0.209990,-0.004963,-0.034856
6,-0.019452,0.071430,0.007873,-0.011133
7,0.013019,-0.285973,-0.074198,-0.009662
8,0.215037,-0.135168,0.307494,-0.130542
9,-0.031971,0.131829,0.021005,-0.069707


In [ ]:
mat_f_df.iloc[20:,:]

,0,1,2,3
20,0.000000,-0.173380,0.080703,-0.061715
21,-0.095194,0.000000,0.037806,-0.136843
22,0.069021,0.172888,0.000000,0.835213
23,-0.108998,-0.878886,2.633786,0.000000


#Full Causal Graph

In [ ]:
matrix_2d_2d_s = mat

In [ ]:
np.savetxt('proposed-full-adj-mat_syn_V2_n.csv', matrix_2d_2d_s, delimiter=",")

In [ ]:
import networkx as nx
G_2d_s = nx.DiGraph()

nodes_2d_s = ["S1(t-5)", "S2(t-5)", "S3(t-5)",  "S4(t-5)",
         "S1(t-4)", "S2(t-4)", "S3(t-4)",  "S4(t-4)",
         "S1(t-3)", "S2(t-3)", "S3(t-3)",  "S4(t-3)",
         "S1(t-2)", "S2(t-2)", "S3(t-2)",  "S4(t-2)",
         "S1(t-1)", "S2(t-1)", "S3(t-1)",  "S4(t-1)",
         "S1(t)", "S2(t)", "S3(t)",  "S4(t)"]
nodes_r_2d_s=["S1(t)", "S2(t)", "S3(t)",  "S4(t)"]
pred_graph_f = np.zeros((4,24))

In [ ]:
for i in range (0, 24):
  G_2d_s.add_node(nodes_2d_s[i],pos=(int(i/4)+1,(i%4)+1))

In [ ]:
for i in range(0, 24):
  for j in range (0, 4):
    if matrix_2d_2d_s[j,i] > 0.3:
      print(i,j, matrix_2d_2d_s[j,i])
      G_2d_s.add_edge(nodes_2d_s[i], nodes_r_2d_s[j], weight=i)
      pred_graph_f[j,i]=1


In [ ]:
pos_2d_s=nx.get_node_attributes(G_2d_s,'pos')
#pos_2d_s

In [ ]:
weights_2d_s = nx.get_edge_attributes(G_2d_s,'weight').values()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 8))
nx.draw(G_2d_s, pos_2d_s, cmap = plt.get_cmap('jet'), edge_cmap= plt.cm.tab20, edge_color=weights_2d_s, with_labels = True, connectionstyle='arc3, rad = 0.3')
#nx.draw_networkx(G, with_labels = True)
plt.show()

In [ ]:
cdt.metrics.SHD(true_full_graph, pred_graph_f)

In [ ]:
true_values = true_full_graph
predictions = pred_graph_f

N = true_values.shape[1]*true_values.shape[0]
accuracy = (true_values == predictions).sum() / N
TP = ((predictions == 1) & (true_values == 1)).sum()
FP = ((predictions == 1) & (true_values == 0)).sum()
TN = ((predictions == 0) & (true_values == 0)).sum()
FN = ((predictions == 0) & (true_values == 1)).sum()
precision = TP / (TP+FP)
recall = TP / (TP + FN)
FDR = FP / (FP + TP)
F1 = 2 * (precision * recall) / (precision + recall)
print('Accuracy: {}, Precision: {}, Recall: {}, FDR: {}, F1 Score: {}'.format(accuracy, precision, recall, FDR,F1))

#Summary Causal Graph

In [ ]:
import networkx as nx
G_2d_s1 = nx.DiGraph()

nodes_2d_s1 = ["S1", "S2", "S3",  "S4"]
nodes_r_2d_s1= ["S1", "S2", "S3",  "S4"]
edges_2d_s1 = []
pred_graph_s1 = np.zeros((4,4))

In [ ]:
for i in range (0, 4):
  G_2d_s1.add_node(nodes_2d_s1[i],pos=(int(i/2)+1,(i%2)+1))

In [ ]:
for i in range(0, 24):
  for j in range (0, 4):
    if matrix_2d_2d_s[j,i] > 0.3:
      print(i,j)
      col = np.round(matrix_2d_2d_s[j,i], 2)
      G_2d_s1.add_edge(nodes_2d_s1[i%4], nodes_r_2d_s1[j], weight=1)
      pred_graph_s1[i%4, j]=1


In [ ]:
pos_2d_s1=nx.get_node_attributes(G_2d_s1,'pos')
pos_2d_s1

{'S1': (1, 1), 'S2': (1, 2), 'S3': (2, 1), 'S4': (2, 2)}

In [ ]:
weights_2d_s1 = nx.get_edge_attributes(G_2d_s1,'weight').values()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl

plt.figure(figsize=(5, 5))
nx.draw(G_2d_s1, pos_2d_s1,  edge_cmap= plt.cm.tab20,  #cmap = plt.get_cmap('jet'),
        font_size=12, node_size=1200, node_color='#c0c0c0', #[30,30,30,30,30],
        edge_color=weights_2d_s1, with_labels = True, connectionstyle='arc3, rad = 0.3')
#nx.draw_networkx(G, with_labels = True)
plt.colorbar(mpl.cm.ScalarMappable(norm=mpl.colors.Normalize(0.3, 1), cmap=plt.cm.tab20),
              orientation='vertical', label='Edge Strength')
plt.show()

In [ ]:
cdt.metrics.SHD(true_graph, pred_graph_s1)

In [ ]:
true_values = true_graph
predictions = pred_graph_s1

N = true_values.shape[1]*true_values.shape[0]
accuracy = (true_values == predictions).sum() / N
TP = ((predictions == 1) & (true_values == 1)).sum()
FP = ((predictions == 1) & (true_values == 0)).sum()
TN = ((predictions == 0) & (true_values == 0)).sum()
FN = ((predictions == 0) & (true_values == 1)).sum()
precision = TP / (TP+FP)
recall = TP / (TP + FN)
FDR = FP / (FP + TP)
F1 = 2 * (precision * recall) / (precision + recall)
print('Accuracy: {}, Precision: {}, Recall: {}, FDR: {}, F1 Score: {}'.format(accuracy, precision, recall, FDR,F1))

# Non-stationarity test

####ADF Test for Stationarity

Null hypothesis: The time series has a unit root and is not stationary.

If failed to be rejected, it suggests the time series is not stationarity.

If p-values is higher than 0.05 alpha level, we cannot reject the null hypothesis. So the time series is not stationary.

In [ ]:
#ADF Test for non-stationarity
def adf_test(data_df):
    test_stat, p_val = [], []
    cv_1pct, cv_5pct, cv_10pct = [], [], []
    stationary = []
    for c in data_df.columns:
        adf_res = adfuller(data_df[c].dropna(), autolag='AIC')
        test_stat.append(adf_res[0])
        p_val.append(adf_res[1])
        cv_1pct.append(adf_res[4]['1%'])
        cv_5pct.append(adf_res[4]['5%'])
        cv_10pct.append(adf_res[4]['10%'])
        if adf_res[1] > 0.05:
          stationary.append('No')
        else:
          stationary.append('Yes')
    adf_res_df = pd.DataFrame({'Test statistic': test_stat,
                               'p-value': p_val,
                               'Critical value - 1%': cv_1pct,
                               'Critical value - 5%': cv_5pct,
                               'Critical value - 10%': cv_10pct,
                               'Stationary': stationary},
                             index=data_df.columns).T
    adf_res_df = adf_res_df.round(4)
    return adf_res_df

In [ ]:
adf_test(data)

,S1,S2,S3,S4
Test statistic,-2.044424,-3.35226,-2.112031,-4.135683
p-value,0.26744,0.012698,0.239705,0.000845
Critical value - 1%,-3.431662,-3.431667,-3.431665,-3.431666
Critical value - 5%,-2.86212,-2.862122,-2.862121,-2.862121
Critical value - 10%,-2.567079,-2.56708,-2.567079,-2.567079
Stationary,No,Yes,No,Yes


#### KPSS Test for Stationary

Null hypothesis: The time series is stationary.

If the p-value is less than 0.05 alpha level, we can reject the null hypothesis and derive that the time series is not stationary.

In [ ]:
def kpss_test(data_df):
    test_stat, p_val = [], []
    cv_1pct, cv_2p5pct, cv_5pct, cv_10pct = [], [], [], []
    stationary = []
    for c in data_df.columns:
        kpss_res = kpss(data_df[c].dropna())#, regression='ct')
        test_stat.append(kpss_res[0])
        p_val.append(kpss_res[1])
        cv_1pct.append(kpss_res[3]['1%'])
        cv_2p5pct.append(kpss_res[3]['2.5%'])
        cv_5pct.append(kpss_res[3]['5%'])
        cv_10pct.append(kpss_res[3]['10%'])
        if kpss_res[1] > 0.05:
          stationary.append('Yes')
        else:
          stationary.append('No')
    kpss_res_df = pd.DataFrame({'Test statistic': test_stat,
                               'p-value': p_val,
                               'Critical value - 1%': cv_1pct,
                               'Critical value - 2.5%': cv_2p5pct,
                               'Critical value - 5%': cv_5pct,
                               'Critical value - 10%': cv_10pct,
                               'Stationary': stationary},
                             index=data_df.columns).T
    kpss_res_df = kpss_res_df.round(4)
    return kpss_res_df

In [ ]:
kpss_test(data)

,S1,S2,S3,S4
Test statistic,2.887966,2.38459,2.912489,2.638319
p-value,0.01,0.01,0.01,0.01
Critical value - 1%,0.739,0.739,0.739,0.739
Critical value - 2.5%,0.574,0.574,0.574,0.574
Critical value - 5%,0.463,0.463,0.463,0.463
Critical value - 10%,0.347,0.347,0.347,0.347
Stationary,No,No,No,No
